In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np # for trigger
mp_drawing = mp.solutions.drawing_utils #
mp_pose = mp.solutions.pose #pose estimation model

In [3]:
#calculate angle of 3 points
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
def elbow_depth_from_x(shoulder_x, elbow_x):
    return abs(elbow_x - shoulder_x)  # Negative = elbow deeper (good)

def normalized_elbow_depth(shoulder, elbow, hip):
    torso_length = abs(shoulder[1] - hip[1])
    depth = elbow[1] - shoulder[1]
    return depth / torso_length if torso_length != 0 else 0

def is_elbow_below_shoulder(shoulder, elbow):
    return elbow[1] > shoulder[1]

DEPTH_THRESHOLD = -0.005  # Adjust based on camera setup and user size

def is_depth_good(x_depth, threshold=DEPTH_THRESHOLD):
    return x_depth < threshold  # Negative z-depth is better


In [6]:
# Mediapipe setup
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Get relevant coordinates (right side)
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * image.shape[1],
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * image.shape[0]]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * image.shape[1],
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * image.shape[0]]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * image.shape[1],
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * image.shape[0]]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x * image.shape[1],
                   landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image.shape[0]]

            # Compute metrics
            angle = calculate_angle(shoulder, elbow, wrist)
            is_deep = is_elbow_below_shoulder(shoulder, elbow)
            depth_norm = normalized_elbow_depth(shoulder, elbow, hip)

            # Get x coordinates (depth)
            shoulder_x = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x
            elbow_x = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x
            x_depth = elbow_depth_from_x(shoulder_x, elbow_x)
            depth_quality = is_depth_good(x_depth)

            # Z-based depth (positive = forward, negative = behind)
            #z_depth = elbow_depth_from_z(shoulder_z, elbow_z)
            #depth_quality = is_depth_good(z_depth)

            # Display results
            
            cv2.putText(image, f'Elbow Angle: {angle:.1f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
            #cv2.putText(image, f'Elbow Below Shoulder: {"Yes" if is_deep else "No"}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0) if is_deep else (0, 0, 255), 2)
            #cv2.putText(image, f'Norm. Depth: {depth_norm:.2f}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 200, 0), 2)
            depth_color = (0, 255, 0) if depth_quality else (0, 0, 255)
            cv2.putText(image, f'X-Depth: {x_depth:.3f} ({ "Good" if depth_quality else "Bad" })',
            (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, depth_color, 2)
            
            # Draw pose landmarks
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the image
        cv2.imshow('Bench Press Elbow Depth Analysis', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()